### Collisional-radiative model for Argon Plasma
#### ROM Basis

In [ ]:
# Check these global settings first:
GLOBAL_SET = {
  # Matplotlib style (None or string)
  "mpl_style": "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle",
  # Path to 'romar' package
  "path_to_romar": "/home/zanardi/Codes/ML/ROMAr/romar/",
  # Path to Argon thermochemical database
  "path_to_dtb": "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/",
  # Path to simulated run
  "path_to_run": "/home/zanardi/Codes/ML/ROMAr/runs/run01/"
}

In [ ]:
import os
import sys
import numpy as np
import dill as pickle

In [ ]:
import matplotlib.pyplot as plt
style = GLOBAL_SET["mpl_style"]
if ((style is not None) and os.path.exists(style)):
  plt.style.use(style)

In [ ]:
sys.path.append(GLOBAL_SET["path_to_romar"])
from romar import env
from romar import utils
from romar.systems import BoxAd
from romar import postproc as pp

Set enviroment

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

Set inputs

In [ ]:
paths = {
  # > ROM basis
  "basis": GLOBAL_SET["path_to_run"] + "/max_mom_2/models/cobras/basis_varimax.p",
  # > Path to solutions folder
  "data": GLOBAL_SET["path_to_run"] + "/data/test/",
  # > Output folder
  "out": "./figs/visual_basis/"
}
# Test cases names
cases = ["lrho_lT", "lrho_hT", "hrho_lT", "hrho_hT", "mrho_mT"]
# Number of ROM dimensions
rdim = 7
# Final time for integration
tend = 1e-3
# Labels for plotting
labels = {
  "Th": "$T_h$",
  "Te": "$T_e$",
  "Ar": "\\text{Ar}",
  "Arp": "\\text{Ar$^+$}",
  "em": "\\text{e$^-$}"
}

In [ ]:
os.makedirs(paths["out"], exist_ok=True)

Initialize 0D thermochemical system

In [ ]:
system = BoxAd(
  species={k: GLOBAL_SET["path_to_dtb"] + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=GLOBAL_SET["path_to_dtb"] + "/rates/kin_fit.p",
  rad_dtb=GLOBAL_SET["path_to_dtb"] + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)

Load ROM basis

In [ ]:
with open(paths["basis"], "rb") as file:
  basis = pickle.load(file)

In [ ]:
system.rom.build(
  phi=basis["phi"][rdim],
  psi=basis["psi"][rdim],
  **{k: basis[k] for k in ("mask", "xref", "xscale")}
)

> Plot loadings

In [ ]:
for name in ("phi", "psi"):
  for i in range(rdim):
    nb = str(i+1)
    b = basis[name][rdim][:,i]
    pp.plot_dist_2d(
      x=np.arange(len(b)),
      y=b,
      labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % (name, nb)],
      scales=["linear", "linear"],
      markersize=1,
      figname=paths["out"] + f"/{name}_{nb.zfill(2)}",
      save=True,
      show=True
    )